# Imports

In [45]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Concatenate, UpSampling2D, \
                                    BatchNormalization, Activation
from tensorflow.keras.models import Model
import cv2
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split

# Load Data

In [46]:
def read_image(file_path):
    image = cv2.imread(file_path)
    image = cv2.resize(image, (64, 64))
    image_array = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image_array

def read_mask(file_path):
    mask = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (64, 64))
    return mask


In [47]:
data_dir = 'ds/people_segmentation'
images_file_paths = glob(data_dir + '/images/*.jpg')
masks_file_paths = glob(data_dir + '/masks/*.png')
# dataset = file_paths.map(load_image_and_mask)

In [48]:
images = []
for image_path in images_file_paths:
  images.append(read_image(image_path))
images = np.array([images])

In [49]:
masks = []
for mask_path in masks_file_paths:
  masks.append(read_mask(mask_path))
masks = np.array([masks])

In [50]:
images = images.reshape(images[0].shape)
masks = masks.reshape(masks[0].shape)

In [51]:
print(images.shape)
print(masks.shape)

(5678, 64, 64, 3)
(5678, 64, 64)


# Split Data

In [52]:
train_x, test_x = train_test_split(images, test_size=0.1, random_state=42)
train_y, test_y = train_test_split(masks, test_size=0.1, random_state=42)

# Define the model

In [53]:
def residual_block(inputs, filters_count):
  x_skip = Conv2D(filters_count, 1, padding = 'same')(inputs)
  x_skip = BatchNormalization()(x_skip)

  x = Conv2D(filters_count, 3, padding = 'same')(inputs)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  x = Conv2D(filters_count, 4, padding = 'same')(x)
  x = BatchNormalization()(x)

  x = Activation('relu')(x + x_skip)

  return x

In [54]:
# def inception_conv_block(inputs, filters_count):
#   x1 = Conv2D(filters_count, 3, padding = "same", dilation_rate=3)(inputs)
#   x1 = BatchNormalization()(x1)
#   x1 = Activation('relu')(x1)

#   x2 = Conv2D(filters_count, 3, padding = "same", dilation_rate=6)(inputs)
#   x2 = BatchNormalization()(x2)
#   x2 = Activation('relu')(x2)

#   x3 = Conv2D(filters_count, 3, padding = "same", dilation_rate=9)(inputs)
#   x3 = BatchNormalization()(x3)
#   x3 = Activation('relu')(x3)

#   x_concat = Concatenate([x1, x2, x3])
#   x_concat = Conv2D(filters_count, 1, padding = "same")(x_concat)
#   x_concat = BatchNormalization()(x_concat)
#   x_concat = Activation('relu')(x_concat)

#   return x_concat

In [55]:
def u_model(input_shape):
    inputs = Input(input_shape)
    
    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    ##
    conv3 = residual_block(pool2, 256)
    # conv3 = inception_conv_block(conv3, 256)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)

    # Decoder
    up4 = UpSampling2D(size=(2, 2))(conv3)
    up4 = Conv2D(128, 2, activation='relu', padding='same')(up4)
    merge4 = Concatenate()([conv2, up4])
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(merge4)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)

    up5 = UpSampling2D(size=(2, 2))(conv4)
    up5 = Conv2D(64, 2, activation='relu', padding='same')(up5)
    merge5 = Concatenate()([conv1, up5])
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(merge5)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)

    # Output layer
    outputs = Conv2D(1, 1, activation='sigmoid')(conv5)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [56]:
model = u_model((64, 64, 3))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [57]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_50 (Conv2D)              (None, 64, 64, 64)   1792        input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_51 (Conv2D)              (None, 64, 64, 64)   36928       conv2d_50[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_8 (MaxPooling2D)  (None, 32, 32, 64)   0           conv2d_51[0][0]                  
____________________________________________________________________________________________

In [58]:
model.fit(train_x, train_y, epochs=15)

Epoch 1/15
160/160 [==============================] - 111s 685ms/step - loss: 0.4980 - accuracy: 0.7820
Epoch 2/15
160/160 [==============================] - 110s 685ms/step - loss: 0.3730 - accuracy: 0.8353
Epoch 3/15
160/160 [==============================] - 110s 685ms/step - loss: 0.3354 - accuracy: 0.8526
Epoch 4/15
160/160 [==============================] - 110s 687ms/step - loss: 0.3117 - accuracy: 0.8650
Epoch 5/15
160/160 [==============================] - 110s 689ms/step - loss: 0.3044 - accuracy: 0.8684
Epoch 6/15
160/160 [==============================] - 110s 689ms/step - loss: 0.2798 - accuracy: 0.8798
Epoch 7/15
160/160 [==============================] - 110s 689ms/step - loss: 0.2649 - accuracy: 0.8869
Epoch 8/15
160/160 [==============================] - 110s 690ms/step - loss: 0.2575 - accuracy: 0.8906
Epoch 9/15
160/160 [==============================] - 110s 689ms/step - loss: 0.2512 - accuracy: 0.8933
Epoch 10/15
160/160 [==============================] - 110s 689m

In [59]:
loss, accuracy = model.evaluate(test_x, test_y)

print(f"Loss: {loss}")
print(f"Accuracy: {round(accuracy*100, 2)}%")

18/18 [==============================] - 11s 554ms/step - loss: 0.2533 - accuracy: 0.8954
Loss: 0.2533043920993805
Accuracy: 89.54%


In [60]:
model.save('back_removal.h5')